# **Code de reproduction de l'article :** How transferable are features in deep neural networks?

In [1]:
%load_ext autoreload
%autoreload 2
import os
import json
from tqdm import tqdm
import torch 
import torch.nn as nn
import models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import Adam
print(f"device avaible : {'cuda' if torch.cuda.is_available() else 'cpu'}")

device avaible : cuda


# **Dataset**

Création de deux datasets distincts : 
- A contient les chiffres de 0 à 4
- B contient les chiffres de 5 à 9 

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train.values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0 
X_test = torch.tensor(X_test.values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# création des datasets A (0-4) et B (5-9)
def create_subset(X, y, class_range):
    indices = [i for i, label in enumerate(y) if label in class_range]
    return Subset(torch.utils.data.TensorDataset(X, y), indices)

train_set_A = create_subset(X_train, y_train, range(0, 5))
train_set_B = create_subset(X_train, y_train, range(5, 10))

test_set_A = create_subset(X_test, y_test, range(0, 5))
test_set_B = create_subset(X_test, y_test, range(5, 10))


train_loader_A = DataLoader(train_set_A, batch_size=64, shuffle=True)
train_loader_B = DataLoader(train_set_B, batch_size=64, shuffle=True)

test_loader_A = DataLoader(test_set_A, batch_size=64, shuffle=False)
test_loader_B = DataLoader(test_set_B, batch_size=64, shuffle=False)

# **Entraînement des modèles de bases**

In [5]:
loss_fn = nn.CrossEntropyLoss()
path = "./resultats/"

resA  = []
resB = []
print("Entrainement du modèle A")
print()
for _ in range(10): 
    modelA = models.CNN()
    optimizerA = Adam(modelA.parameters())
    modelA.train_model(train_loader_A,test_loader_A, loss_fn, n_epochs=2, optimizer=optimizerA)
    resA.append(modelA.test(test_loader_A))
torch.save(modelA, "./models/baseA.torch")

print()
print("Entrainement du modèle B")
print()
for _ in range(10): 
    modelB = models.CNN()
    optimizerB= Adam(modelB.parameters())
    modelB.train_model(train_loader_B,test_loader_B, loss_fn, n_epochs=2, optimizer=optimizerB)
    resB.append(modelB.test(test_loader_B))
torch.save(modelB, "./models/baseB.torch")

with open(os.path.join(path, "base.json"), 'a') as file:
        json.dump({"model A" : resA}, file)
        json.dump({"model B" : resB}, file)

Entrainement du modèle A

Epoch 1, Loss: 0.2022
Test set: Avg. loss: 0.0312, Accuracy: 6980/7051 (98.99%)
Epoch 2, Loss: 0.0221
Test set: Avg. loss: 0.0201, Accuracy: 7008/7051 (99.39%)
Test set: Avg. loss: 0.0201, Accuracy: 7008/7051 (99.39%)
Epoch 1, Loss: 0.1696
Test set: Avg. loss: 0.0361, Accuracy: 6967/7051 (98.81%)
Epoch 2, Loss: 0.0235
Test set: Avg. loss: 0.0172, Accuracy: 7008/7051 (99.39%)
Test set: Avg. loss: 0.0172, Accuracy: 7008/7051 (99.39%)
Epoch 1, Loss: 0.2140
Test set: Avg. loss: 0.0339, Accuracy: 6984/7051 (99.05%)
Epoch 2, Loss: 0.0276
Test set: Avg. loss: 0.0187, Accuracy: 7004/7051 (99.33%)
Test set: Avg. loss: 0.0187, Accuracy: 7004/7051 (99.33%)
Epoch 1, Loss: 0.2647
Test set: Avg. loss: 0.0361, Accuracy: 6968/7051 (98.82%)
Epoch 2, Loss: 0.0251
Test set: Avg. loss: 0.0209, Accuracy: 7008/7051 (99.39%)
Test set: Avg. loss: 0.0209, Accuracy: 7008/7051 (99.39%)
Epoch 1, Loss: 0.1695
Test set: Avg. loss: 0.0328, Accuracy: 6984/7051 (99.05%)
Epoch 2, Loss: 0.0261


FileNotFoundError: [Errno 2] No such file or directory: './resultats/base.json'

# **Réentrainement des modèles**

Nous réalisons l'expérience décrite dans le papier. Nous prenons le modèle B comme référence. Nous allons faire une boucle pour itérer sur le nombre de couche du réseau. Commençons pas le cas ou les paramètres des couches copiées sont gelés. Les paramètres du classifieur sont toujours réappris

## modèles BNB

In [7]:
path = "./resultats/"


basemodel = torch.load("./models/baseB.torch")
loss_fn = nn.CrossEntropyLoss()


final = {}
for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        for param in newmodel.model[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "BNB.json"), 'a') as file:
    json.dump(final, file)

/tmp/ipykernel_60135/164737328.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseB.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it/

Epoch 1, Loss: 0.1872
Test set: Avg. loss: 0.0595, Accuracy: 6823/6949 (98.19%)
Epoch 2, Loss: 0.0403
Test set: Avg. loss: 0.0285, Accuracy: 6883/6949 (99.05%)
Test set: Avg. loss: 0.0285, Accuracy: 6883/6949 (99.05%)
Epoch 1, Loss: 0.2729
Test set: Avg. loss: 0.0411, Accuracy: 6860/6949 (98.72%)
Epoch 2, Loss: 0.0390
Test set: Avg. loss: 0.0581, Accuracy: 6819/6949 (98.13%)
Test set: Avg. loss: 0.0581, Accuracy: 6819/6949 (98.13%)
Epoch 1, Loss: 0.3495
Test set: Avg. loss: 0.0613, Accuracy: 6814/6949 (98.06%)
Epoch 2, Loss: 0.0515
Test set: Avg. loss: 0.0413, Accuracy: 6852/6949 (98.60%)
Test set: Avg. loss: 0.0413, Accuracy: 6852/6949 (98.60%)
Epoch 1, Loss: 0.2498
Test set: Avg. loss: 0.0599, Accuracy: 6824/6949 (98.20%)
Epoch 2, Loss: 0.0423
Test set: Avg. loss: 0.0440, Accuracy: 6854/6949 (98.63%)
Test set: Avg. loss: 0.0440, Accuracy: 6854/6949 (98.63%)
Epoch 1, Loss: 0.1914
Test set: Avg. loss: 0.0749, Accuracy: 6778/6949 (97.54%)
Epoch 2, Loss: 0.0450
Test set: Avg. loss: 0.034

Etape :  12%|█▎        | 1/8 [02:16<15:52, 136.08s/it]

Test set: Avg. loss: 0.0383, Accuracy: 6863/6949 (98.76%)
Epoch 1, Loss: 0.2033
Test set: Avg. loss: 0.0459, Accuracy: 6843/6949 (98.47%)
Epoch 2, Loss: 0.0402
Test set: Avg. loss: 0.0364, Accuracy: 6866/6949 (98.81%)
Test set: Avg. loss: 0.0364, Accuracy: 6866/6949 (98.81%)
Epoch 1, Loss: 0.2317
Test set: Avg. loss: 0.0577, Accuracy: 6819/6949 (98.13%)
Epoch 2, Loss: 0.0439
Test set: Avg. loss: 0.0366, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0366, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.2346
Test set: Avg. loss: 0.0431, Accuracy: 6862/6949 (98.75%)
Epoch 2, Loss: 0.0390
Test set: Avg. loss: 0.0433, Accuracy: 6866/6949 (98.81%)
Test set: Avg. loss: 0.0433, Accuracy: 6866/6949 (98.81%)
Epoch 1, Loss: 0.3035
Test set: Avg. loss: 0.0439, Accuracy: 6864/6949 (98.78%)
Epoch 2, Loss: 0.0416
Test set: Avg. loss: 0.0350, Accuracy: 6874/6949 (98.92%)
Test set: Avg. loss: 0.0350, Accuracy: 6874/6949 (98.92%)
Epoch 1, Loss: 0.2636
Test set: Avg. loss: 0.0479, Accuracy: 6851/6949

Etape :  25%|██▌       | 2/8 [04:21<13:00, 130.08s/it]

...

Test set: Avg. loss: 0.0585, Accuracy: 6818/6949 (98.11%)
Epoch 1, Loss: 0.2445
Test set: Avg. loss: 0.1027, Accuracy: 6695/6949 (96.34%)
Epoch 2, Loss: 0.0651
Test set: Avg. loss: 0.0446, Accuracy: 6851/6949 (98.59%)
Test set: Avg. loss: 0.0446, Accuracy: 6851/6949 (98.59%)
Epoch 1, Loss: 0.2259
Test set: Avg. loss: 0.0479, Accuracy: 6846/6949 (98.52%)
Epoch 2, Loss: 0.0523
Test set: Avg. loss: 0.0441, Accuracy: 6851/6949 (98.59%)
Test set: Avg. loss: 0.0441, Accuracy: 6851/6949 (98.59%)
Epoch 1, Loss: 0.2469
Test set: Avg. loss: 0.0921, Accuracy: 6740/6949 (96.99%)
Epoch 2, Loss: 0.0764
Test set: Avg. loss: 0.0805, Accuracy: 6777/6949 (97.52%)
Test set: Avg. loss: 0.0805, Accuracy: 6777/6949 (97.52%)
Epoch 1, Loss: 0.2486
Test set: Avg. loss: 0.0765, Accuracy: 6781/6949 (97.58%)
Epoch 2, Loss: 0.0692
Test set: Avg. loss: 0.0506, Accuracy: 6842/6949 (98.46%)
Test set: Avg. loss: 0.0506, Accuracy: 6842/6949 (98.46%)
Epoch 1, Loss: 0.2223
Test set: Avg. loss: 0.0512, Accuracy: 6825/6949

Etape :  88%|████████▊ | 7/8 [14:35<02:03, 123.09s/it]

Test set: Avg. loss: 0.0467, Accuracy: 6852/6949 (98.60%)
Epoch 1, Loss: 0.1449
Test set: Avg. loss: 0.0720, Accuracy: 6788/6949 (97.68%)
Epoch 2, Loss: 0.0419
Test set: Avg. loss: 0.0356, Accuracy: 6859/6949 (98.70%)
Test set: Avg. loss: 0.0356, Accuracy: 6859/6949 (98.70%)
Epoch 1, Loss: 0.1723
Test set: Avg. loss: 0.0642, Accuracy: 6816/6949 (98.09%)
Epoch 2, Loss: 0.0457
Test set: Avg. loss: 0.0400, Accuracy: 6859/6949 (98.70%)
Test set: Avg. loss: 0.0400, Accuracy: 6859/6949 (98.70%)
Epoch 1, Loss: 0.1418
Test set: Avg. loss: 0.0505, Accuracy: 6845/6949 (98.50%)
Epoch 2, Loss: 0.0453
Test set: Avg. loss: 0.0473, Accuracy: 6849/6949 (98.56%)
Test set: Avg. loss: 0.0473, Accuracy: 6849/6949 (98.56%)
Epoch 1, Loss: 0.2515
Test set: Avg. loss: 0.0765, Accuracy: 6757/6949 (97.24%)
Epoch 2, Loss: 0.0767
Test set: Avg. loss: 0.1019, Accuracy: 6698/6949 (96.39%)
Test set: Avg. loss: 0.1019, Accuracy: 6698/6949 (96.39%)
Epoch 1, Loss: 0.1853
Test set: Avg. loss: 0.0592, Accuracy: 6809/6949

Etape : 100%|██████████| 8/8 [16:35<00:00, 124.47s/it]

Test set: Avg. loss: 0.0439, Accuracy: 6859/6949 (98.70%)


## Modèles AnB

In [8]:
path = "./resultats/"


basemodel = torch.load("./models/baseA.torch")
loss_fn = nn.CrossEntropyLoss()

final = {}
for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        for param in newmodel.model[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "ANB.json"), 'a') as file:
    json.dump(final, file)

/tmp/ipykernel_60135/159931443.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it/

Epoch 1, Loss: 0.2579
Test set: Avg. loss: 0.0553, Accuracy: 6829/6949 (98.27%)
Epoch 2, Loss: 0.0394
Test set: Avg. loss: 0.0466, Accuracy: 6840/6949 (98.43%)
Test set: Avg. loss: 0.0466, Accuracy: 6840/6949 (98.43%)
Epoch 1, Loss: 0.2476
Test set: Avg. loss: 0.0470, Accuracy: 6842/6949 (98.46%)
Epoch 2, Loss: 0.0408
Test set: Avg. loss: 0.0360, Accuracy: 6871/6949 (98.88%)
Test set: Avg. loss: 0.0360, Accuracy: 6871/6949 (98.88%)
Epoch 1, Loss: 0.2727
Test set: Avg. loss: 0.0663, Accuracy: 6808/6949 (97.97%)
Epoch 2, Loss: 0.0408
Test set: Avg. loss: 0.0444, Accuracy: 6852/6949 (98.60%)
Test set: Avg. loss: 0.0444, Accuracy: 6852/6949 (98.60%)
Epoch 1, Loss: 0.2514
Test set: Avg. loss: 0.0437, Accuracy: 6843/6949 (98.47%)
Epoch 2, Loss: 0.0405
Test set: Avg. loss: 0.0637, Accuracy: 6800/6949 (97.86%)
Test set: Avg. loss: 0.0637, Accuracy: 6800/6949 (97.86%)
Epoch 1, Loss: 0.2312
Test set: Avg. loss: 0.0526, Accuracy: 6847/6949 (98.53%)
Epoch 2, Loss: 0.0378
Test set: Avg. loss: 0.038

Etape :  12%|█▎        | 1/8 [02:05<14:39, 125.61s/it]

Test set: Avg. loss: 0.0547, Accuracy: 6826/6949 (98.23%)
Epoch 1, Loss: 0.1860
Test set: Avg. loss: 0.0764, Accuracy: 6775/6949 (97.50%)
Epoch 2, Loss: 0.0419
Test set: Avg. loss: 0.0420, Accuracy: 6862/6949 (98.75%)
Test set: Avg. loss: 0.0420, Accuracy: 6862/6949 (98.75%)
Epoch 1, Loss: 0.2755
Test set: Avg. loss: 0.0617, Accuracy: 6819/6949 (98.13%)
Epoch 2, Loss: 0.0446
Test set: Avg. loss: 0.0555, Accuracy: 6828/6949 (98.26%)
Test set: Avg. loss: 0.0555, Accuracy: 6828/6949 (98.26%)
Epoch 1, Loss: 0.2904
Test set: Avg. loss: 0.0643, Accuracy: 6801/6949 (97.87%)
Epoch 2, Loss: 0.0422
Test set: Avg. loss: 0.0353, Accuracy: 6870/6949 (98.86%)
Test set: Avg. loss: 0.0353, Accuracy: 6870/6949 (98.86%)
Epoch 1, Loss: 0.2366
Test set: Avg. loss: 0.0503, Accuracy: 6832/6949 (98.32%)
Epoch 2, Loss: 0.0406
Test set: Avg. loss: 0.0336, Accuracy: 6879/6949 (98.99%)
Test set: Avg. loss: 0.0336, Accuracy: 6879/6949 (98.99%)
Epoch 1, Loss: 0.2957
Test set: Avg. loss: 0.0458, Accuracy: 6844/6949

Etape :  25%|██▌       | 2/8 [04:10<12:29, 124.92s/it]

...

Test set: Avg. loss: 0.0717, Accuracy: 6787/6949 (97.67%)
Epoch 1, Loss: 0.1796
Test set: Avg. loss: 0.0544, Accuracy: 6829/6949 (98.27%)
Epoch 2, Loss: 0.0459
Test set: Avg. loss: 0.0610, Accuracy: 6831/6949 (98.30%)
Test set: Avg. loss: 0.0610, Accuracy: 6831/6949 (98.30%)
Epoch 1, Loss: 0.2186
Test set: Avg. loss: 0.0628, Accuracy: 6810/6949 (98.00%)
Epoch 2, Loss: 0.0469
Test set: Avg. loss: 0.0388, Accuracy: 6858/6949 (98.69%)
Test set: Avg. loss: 0.0388, Accuracy: 6858/6949 (98.69%)
Epoch 1, Loss: 0.2101
Test set: Avg. loss: 0.0619, Accuracy: 6814/6949 (98.06%)
Epoch 2, Loss: 0.0522
Test set: Avg. loss: 0.0456, Accuracy: 6854/6949 (98.63%)
Test set: Avg. loss: 0.0456, Accuracy: 6854/6949 (98.63%)
Epoch 1, Loss: 0.2313
Test set: Avg. loss: 0.0617, Accuracy: 6828/6949 (98.26%)
Epoch 2, Loss: 0.0517
Test set: Avg. loss: 0.0405, Accuracy: 6869/6949 (98.85%)
Test set: Avg. loss: 0.0405, Accuracy: 6869/6949 (98.85%)
Epoch 1, Loss: 0.2078
Test set: Avg. loss: 0.0514, Accuracy: 6839/6949

Etape :  88%|████████▊ | 7/8 [14:16<02:01, 121.23s/it]

Test set: Avg. loss: 0.0446, Accuracy: 6851/6949 (98.59%)
Epoch 1, Loss: 0.2192
Test set: Avg. loss: 0.0635, Accuracy: 6803/6949 (97.90%)
Epoch 2, Loss: 0.0609
Test set: Avg. loss: 0.0491, Accuracy: 6840/6949 (98.43%)
Test set: Avg. loss: 0.0491, Accuracy: 6840/6949 (98.43%)
Epoch 1, Loss: 0.1598
Test set: Avg. loss: 0.0459, Accuracy: 6842/6949 (98.46%)
Epoch 2, Loss: 0.0457
Test set: Avg. loss: 0.0311, Accuracy: 6884/6949 (99.06%)
Test set: Avg. loss: 0.0311, Accuracy: 6884/6949 (99.06%)
Epoch 1, Loss: 0.1859
Test set: Avg. loss: 0.0519, Accuracy: 6833/6949 (98.33%)
Epoch 2, Loss: 0.0601
Test set: Avg. loss: 0.0466, Accuracy: 6836/6949 (98.37%)
Test set: Avg. loss: 0.0466, Accuracy: 6836/6949 (98.37%)
Epoch 1, Loss: 0.1778
Test set: Avg. loss: 0.0762, Accuracy: 6781/6949 (97.58%)
Epoch 2, Loss: 0.0525
Test set: Avg. loss: 0.0497, Accuracy: 6850/6949 (98.58%)
Test set: Avg. loss: 0.0497, Accuracy: 6850/6949 (98.58%)
Epoch 1, Loss: 0.1755
Test set: Avg. loss: 0.0523, Accuracy: 6824/6949

Etape : 100%|██████████| 8/8 [16:15<00:00, 121.89s/it]

Test set: Avg. loss: 0.0471, Accuracy: 6845/6949 (98.50%)


Cette fois-ci on copie des paramètres mais on ne les gèles pas.

## Modèles BnB+

In [9]:
path = "./resultats/"


basemodel = torch.load("./models/baseB.torch")
loss_fn = nn.CrossEntropyLoss()

final = {}

for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "BNB+.json"), 'a') as file:
    json.dump(final, file)

/tmp/ipykernel_60135/3755875823.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseB.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it

Epoch 1, Loss: 0.3402
Test set: Avg. loss: 0.0676, Accuracy: 6795/6949 (97.78%)
Epoch 2, Loss: 0.0453
Test set: Avg. loss: 0.0360, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0360, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.2144
Test set: Avg. loss: 0.0392, Accuracy: 6856/6949 (98.66%)
Epoch 2, Loss: 0.0415
Test set: Avg. loss: 0.0329, Accuracy: 6871/6949 (98.88%)
Test set: Avg. loss: 0.0329, Accuracy: 6871/6949 (98.88%)
Epoch 1, Loss: 0.2422
Test set: Avg. loss: 0.0509, Accuracy: 6834/6949 (98.35%)
Epoch 2, Loss: 0.0451
Test set: Avg. loss: 0.0343, Accuracy: 6874/6949 (98.92%)
Test set: Avg. loss: 0.0343, Accuracy: 6874/6949 (98.92%)
Epoch 1, Loss: 0.1959
Test set: Avg. loss: 0.0409, Accuracy: 6859/6949 (98.70%)
Epoch 2, Loss: 0.0392
Test set: Avg. loss: 0.0383, Accuracy: 6859/6949 (98.70%)
Test set: Avg. loss: 0.0383, Accuracy: 6859/6949 (98.70%)
Epoch 1, Loss: 0.2855
Test set: Avg. loss: 0.0553, Accuracy: 6827/6949 (98.24%)
Epoch 2, Loss: 0.0413
Test set: Avg. loss: 0.030

Etape :  12%|█▎        | 1/8 [02:06<14:46, 126.71s/it]

Test set: Avg. loss: 0.0724, Accuracy: 6794/6949 (97.77%)
Epoch 1, Loss: 0.2684
Test set: Avg. loss: 0.0425, Accuracy: 6852/6949 (98.60%)
Epoch 2, Loss: 0.0406
Test set: Avg. loss: 0.0416, Accuracy: 6864/6949 (98.78%)
Test set: Avg. loss: 0.0416, Accuracy: 6864/6949 (98.78%)
Epoch 1, Loss: 0.2131
Test set: Avg. loss: 0.0540, Accuracy: 6836/6949 (98.37%)
Epoch 2, Loss: 0.0437
Test set: Avg. loss: 0.0428, Accuracy: 6855/6949 (98.65%)
Test set: Avg. loss: 0.0428, Accuracy: 6855/6949 (98.65%)
Epoch 1, Loss: 0.3295
Test set: Avg. loss: 0.0534, Accuracy: 6832/6949 (98.32%)
Epoch 2, Loss: 0.0396
Test set: Avg. loss: 0.0323, Accuracy: 6881/6949 (99.02%)
Test set: Avg. loss: 0.0323, Accuracy: 6881/6949 (99.02%)
Epoch 1, Loss: 0.2289
Test set: Avg. loss: 0.0468, Accuracy: 6850/6949 (98.58%)
Epoch 2, Loss: 0.0425
Test set: Avg. loss: 0.0357, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0357, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.3060
Test set: Avg. loss: 0.0427, Accuracy: 6849/6949

Etape :  25%|██▌       | 2/8 [04:12<12:38, 126.45s/it]

...

Test set: Avg. loss: 0.0284, Accuracy: 6881/6949 (99.02%)
Epoch 1, Loss: 0.1473
Test set: Avg. loss: 0.0538, Accuracy: 6830/6949 (98.29%)
Epoch 2, Loss: 0.0407
Test set: Avg. loss: 0.0322, Accuracy: 6872/6949 (98.89%)
Test set: Avg. loss: 0.0322, Accuracy: 6872/6949 (98.89%)
Epoch 1, Loss: 0.1353
Test set: Avg. loss: 0.0506, Accuracy: 6841/6949 (98.45%)
Epoch 2, Loss: 0.0360
Test set: Avg. loss: 0.0478, Accuracy: 6847/6949 (98.53%)
Test set: Avg. loss: 0.0478, Accuracy: 6847/6949 (98.53%)
Epoch 1, Loss: 0.1704
Test set: Avg. loss: 0.0505, Accuracy: 6842/6949 (98.46%)
Epoch 2, Loss: 0.0384
Test set: Avg. loss: 0.0300, Accuracy: 6885/6949 (99.08%)
Test set: Avg. loss: 0.0300, Accuracy: 6885/6949 (99.08%)
Epoch 1, Loss: 0.1307
Test set: Avg. loss: 0.0373, Accuracy: 6875/6949 (98.94%)
Epoch 2, Loss: 0.0356
Test set: Avg. loss: 0.0260, Accuracy: 6888/6949 (99.12%)
Test set: Avg. loss: 0.0260, Accuracy: 6888/6949 (99.12%)
Epoch 1, Loss: 0.1773
Test set: Avg. loss: 0.0429, Accuracy: 6857/6949

Etape :  88%|████████▊ | 7/8 [14:48<02:07, 127.07s/it]

Test set: Avg. loss: 0.0319, Accuracy: 6883/6949 (99.05%)
Epoch 1, Loss: 0.1284
Test set: Avg. loss: 0.0469, Accuracy: 6844/6949 (98.49%)
Epoch 2, Loss: 0.0392
Test set: Avg. loss: 0.0324, Accuracy: 6881/6949 (99.02%)
Test set: Avg. loss: 0.0324, Accuracy: 6881/6949 (99.02%)
Epoch 1, Loss: 0.1338
Test set: Avg. loss: 0.0390, Accuracy: 6857/6949 (98.68%)
Epoch 2, Loss: 0.0369
Test set: Avg. loss: 0.0330, Accuracy: 6882/6949 (99.04%)
Test set: Avg. loss: 0.0330, Accuracy: 6882/6949 (99.04%)
Epoch 1, Loss: 0.1272
Test set: Avg. loss: 0.0397, Accuracy: 6866/6949 (98.81%)
Epoch 2, Loss: 0.0362
Test set: Avg. loss: 0.0323, Accuracy: 6889/6949 (99.14%)
Test set: Avg. loss: 0.0323, Accuracy: 6889/6949 (99.14%)
Epoch 1, Loss: 0.1225
Test set: Avg. loss: 0.0417, Accuracy: 6857/6949 (98.68%)
Epoch 2, Loss: 0.0314
Test set: Avg. loss: 0.0348, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0348, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.1102
Test set: Avg. loss: 0.0483, Accuracy: 6840/6949

Etape : 100%|██████████| 8/8 [16:55<00:00, 126.95s/it]

Test set: Avg. loss: 0.0478, Accuracy: 6842/6949 (98.46%)


## Modèle AnB+

In [10]:
path = "./resultats/"


basemodel = torch.load("./models/baseA.torch")
loss_fn = nn.CrossEntropyLoss()

final = {}
for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "ANB+.json"), 'a') as file:
    json.dump(final, file)

/tmp/ipykernel_60135/3058033028.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it

Epoch 1, Loss: 0.1967
Test set: Avg. loss: 0.0577, Accuracy: 6824/6949 (98.20%)
Epoch 2, Loss: 0.0409
Test set: Avg. loss: 0.0302, Accuracy: 6878/6949 (98.98%)
Test set: Avg. loss: 0.0302, Accuracy: 6878/6949 (98.98%)
Epoch 1, Loss: 0.3242
Test set: Avg. loss: 0.0509, Accuracy: 6841/6949 (98.45%)
Epoch 2, Loss: 0.0369
Test set: Avg. loss: 0.0286, Accuracy: 6886/6949 (99.09%)
Test set: Avg. loss: 0.0286, Accuracy: 6886/6949 (99.09%)
Epoch 1, Loss: 0.2323
Test set: Avg. loss: 0.0493, Accuracy: 6843/6949 (98.47%)
Epoch 2, Loss: 0.0425
Test set: Avg. loss: 0.0370, Accuracy: 6857/6949 (98.68%)
Test set: Avg. loss: 0.0370, Accuracy: 6857/6949 (98.68%)
Epoch 1, Loss: 0.1977
Test set: Avg. loss: 0.0651, Accuracy: 6810/6949 (98.00%)
Epoch 2, Loss: 0.0408
Test set: Avg. loss: 0.0613, Accuracy: 6828/6949 (98.26%)
Test set: Avg. loss: 0.0613, Accuracy: 6828/6949 (98.26%)
Epoch 1, Loss: 0.2451
Test set: Avg. loss: 0.0542, Accuracy: 6829/6949 (98.27%)
Epoch 2, Loss: 0.0430
Test set: Avg. loss: 0.035

Etape :  12%|█▎        | 1/8 [02:06<14:43, 126.16s/it]

Test set: Avg. loss: 0.0370, Accuracy: 6862/6949 (98.75%)
Epoch 1, Loss: 0.2429
Test set: Avg. loss: 0.0444, Accuracy: 6854/6949 (98.63%)
Epoch 2, Loss: 0.0412
Test set: Avg. loss: 0.0399, Accuracy: 6861/6949 (98.73%)
Test set: Avg. loss: 0.0399, Accuracy: 6861/6949 (98.73%)
Epoch 1, Loss: 0.3049
Test set: Avg. loss: 0.1070, Accuracy: 6708/6949 (96.53%)
Epoch 2, Loss: 0.0453
Test set: Avg. loss: 0.0291, Accuracy: 6884/6949 (99.06%)
Test set: Avg. loss: 0.0291, Accuracy: 6884/6949 (99.06%)
Epoch 1, Loss: 0.2787
Test set: Avg. loss: 0.0569, Accuracy: 6822/6949 (98.17%)
Epoch 2, Loss: 0.0400
Test set: Avg. loss: 0.0352, Accuracy: 6884/6949 (99.06%)
Test set: Avg. loss: 0.0352, Accuracy: 6884/6949 (99.06%)
Epoch 1, Loss: 0.3040
Test set: Avg. loss: 0.0573, Accuracy: 6825/6949 (98.22%)
Epoch 2, Loss: 0.0401
Test set: Avg. loss: 0.0294, Accuracy: 6881/6949 (99.02%)
Test set: Avg. loss: 0.0294, Accuracy: 6881/6949 (99.02%)
Epoch 1, Loss: 0.2856
Test set: Avg. loss: 0.0557, Accuracy: 6832/6949

Etape :  25%|██▌       | 2/8 [04:12<12:36, 126.14s/it]

...

Test set: Avg. loss: 0.0418, Accuracy: 6861/6949 (98.73%)
Epoch 1, Loss: 0.1293
Test set: Avg. loss: 0.0360, Accuracy: 6871/6949 (98.88%)
Epoch 2, Loss: 0.0327
Test set: Avg. loss: 0.0409, Accuracy: 6866/6949 (98.81%)
Test set: Avg. loss: 0.0409, Accuracy: 6866/6949 (98.81%)
Epoch 1, Loss: 0.1225
Test set: Avg. loss: 0.0634, Accuracy: 6818/6949 (98.11%)
Epoch 2, Loss: 0.0413
Test set: Avg. loss: 0.0339, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0339, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.1242
Test set: Avg. loss: 0.0342, Accuracy: 6879/6949 (98.99%)
Epoch 2, Loss: 0.0410
Test set: Avg. loss: 0.0319, Accuracy: 6877/6949 (98.96%)
Test set: Avg. loss: 0.0319, Accuracy: 6877/6949 (98.96%)
Epoch 1, Loss: 0.1402
Test set: Avg. loss: 0.0345, Accuracy: 6873/6949 (98.91%)
Epoch 2, Loss: 0.0357
Test set: Avg. loss: 0.0492, Accuracy: 6840/6949 (98.43%)
Test set: Avg. loss: 0.0492, Accuracy: 6840/6949 (98.43%)
Epoch 1, Loss: 0.1154
Test set: Avg. loss: 0.0352, Accuracy: 6871/6949

Etape :  88%|████████▊ | 7/8 [14:47<02:06, 126.96s/it]

Test set: Avg. loss: 0.0344, Accuracy: 6882/6949 (99.04%)
Epoch 1, Loss: 0.1434
Test set: Avg. loss: 0.0382, Accuracy: 6860/6949 (98.72%)
Epoch 2, Loss: 0.0360
Test set: Avg. loss: 0.0235, Accuracy: 6896/6949 (99.24%)
Test set: Avg. loss: 0.0235, Accuracy: 6896/6949 (99.24%)
Epoch 1, Loss: 0.1334
Test set: Avg. loss: 0.0420, Accuracy: 6845/6949 (98.50%)
Epoch 2, Loss: 0.0364
Test set: Avg. loss: 0.0340, Accuracy: 6880/6949 (99.01%)
Test set: Avg. loss: 0.0340, Accuracy: 6880/6949 (99.01%)
Epoch 1, Loss: 0.1168
Test set: Avg. loss: 0.0380, Accuracy: 6857/6949 (98.68%)
Epoch 2, Loss: 0.0377
Test set: Avg. loss: 0.0309, Accuracy: 6879/6949 (98.99%)
Test set: Avg. loss: 0.0309, Accuracy: 6879/6949 (98.99%)
Epoch 1, Loss: 0.1035
Test set: Avg. loss: 0.0398, Accuracy: 6862/6949 (98.75%)
Epoch 2, Loss: 0.0343
Test set: Avg. loss: 0.0338, Accuracy: 6871/6949 (98.88%)
Test set: Avg. loss: 0.0338, Accuracy: 6871/6949 (98.88%)
Epoch 1, Loss: 0.1107
Test set: Avg. loss: 0.0430, Accuracy: 6854/6949

Etape : 100%|██████████| 8/8 [16:54<00:00, 126.77s/it]

Test set: Avg. loss: 0.0368, Accuracy: 6870/6949 (98.86%)
